# 1.1 Import libraries, data, and globals

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Get rid of warnings
import warnings
warnings.filterwarnings('ignore')

# General libraries
import requests
import pandas as pd
import string
import time
from bs4 import BeautifulSoup
from pandas.plotting import scatter_matrix
import os 
import string
import random
import numpy as np
import re
import datetime
from dateutil.relativedelta import relativedelta
import category_encoders as ce
import pickle

# Sklearn Specific
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_validate
from sklearn import preprocessing

# sklearn algos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier # SGD
from sklearn.neural_network import MLPClassifier # neural network (multilayer perceptron)

# Bayesian Hyperparameter optimization
from hyperopt import hp, tpe, fmin

# Import data and remove Tony Johnson
fs_df = pd.read_csv("../data/scraped_data/full_stats_v5.csv")
fs_df = fs_df[fs_df['fighter_1'] != "Tony Johnson"]
fs_df = fs_df[fs_df['fighter_2'] != "Tony Johnson"]

fc_df = pd.read_csv("../data/scraped_data/fighter_char_v3.csv")
fc_df = fc_df[fc_df['name'] != "Tony Johnson"]

# 1.2 Functions

In [2]:
# Create dictionary with each fighter having a pd.df with all of their fights
def generate_fighters_dict():
    cols = two_pf.columns.tolist()
    fighters = {}
    for index, row in two_pf.iterrows():
        fighter = row["a_fighter"]
        if fighter not in fighters:
            fighters[fighter] = [row]
        else:
            fighters[fighter].append(row)
    for fighter in fighters:
        rows_list = fighters[fighter]
        fighters[fighter] = pd.DataFrame(columns=cols, data=rows_list)
    return fighters

# Gets filtered fighter information
def get_fighter_info(past_df, is_fighter, lb_num, is_exact):
    if is_exact:
        latest_df = past_df.head(lb_num)
    else:
        latest_df = past_df
        
    latest_lb_cum = latest_df.mean(skipna=True, numeric_only=True).to_dict()
    new_dict = {}
    if is_fighter:
        for var in latest_lb_cum:
            if var[0] == "f":
                new_dict[var] = latest_lb_cum[var]
    else:
        for var in latest_lb_cum:
            if var[0] == "f":
                new_var = var.replace("f_", "o_")
                new_dict[new_var] = latest_lb_cum[var]
    return(new_dict)

# Generates cumulative df
def generate_cum_df(fighters_dict, lb_num, is_exact):
    
    cumulative_df = pd.DataFrame(columns = two_pf.columns)
    
    for index, row in two_pf.iterrows():
        fighter_name = row["a_fighter"]
        opponent_name = row["a_opponent"]
        fight_date = row["date"]
        fighter_df = fighters_dict[fighter_name]
        opponent_df = fighters_dict[opponent_name]
        
        past_fighter_df = fighter_df.loc[fighter_df['date'] < fight_date]
        if past_fighter_df.shape[0] < lb_num:
            continue
        past_opponent_df = opponent_df.loc[opponent_df['date'] < fight_date]
        if past_opponent_df.shape[0] < lb_num:
            continue
        fighter_info = get_fighter_info(past_fighter_df, True, lb_num, is_exact)
        opponent_info = get_fighter_info(past_opponent_df, False, lb_num, is_exact)
        all_info = dict(fighter_info, **opponent_info)
        all_info['a_fighter'] = row['a_fighter']
        all_info['a_opponent'] = row['a_opponent']
        all_info['a_winner'] = row['a_winner']
        all_info['date'] = row['date']
        all_info['result'] = row['result']
    
        # Merge fight characterestics
        fc_index = 0
        f_init = "fo"
        fighters_list = [fighter_name,opponent_name]
        
        for fighter in fighters_list:
            fc_index_r = fc_index%2
            
            # Merge in height, reach, and stance
            all_info['{0}_height'.format(f_init[fc_index_r])] = fc_df.loc[fc_df['full_name'] == fighters_list[fc_index_r]]['height'].item()
            all_info['{0}_reach'.format(f_init[fc_index_r])] = fc_df.loc[fc_df['full_name'] == fighters_list[fc_index_r]]['reach'].item()
            all_info['{0}_stance'.format(f_init[fc_index_r])] = fc_df.loc[fc_df['full_name'] == fighters_list[fc_index_r]]['stance'].item()
            
            #Merge in age of fighter
            DOB = fc_df.loc[fc_df['full_name'] == fighters_list[fc_index_r]]['DOB'].item()
            if(DOB == "--"):
                break
            age = relativedelta(fight_date, DOB).years
            all_info['{0}_age'.format(f_init[fc_index_r])] = age

            fc_index += 1
            
        
        # Convert dict to pdf
        all_info_df = pd.DataFrame(all_info, index=[0])
        
        cumulative_df = pd.concat([cumulative_df, all_info_df])
        
    return(cumulative_df)


# 2.1 Data wrangling for full_stats

#### Add Totals

In [3]:
fs_df = fs_df.fillna(0) # Fill NaN with 0

# Calc time of fight in seconds
fs_df['duration'] = fs_df.apply(lambda x: (int(x['end_round'])-1)*60*5 + int(x['end_time'].split(":")[0])*60 + int(x['end_time'].split(":")[1]) , axis = 1)

# Convert date to date object
fs_df['date'] = fs_df.apply(lambda x: datetime.datetime.strptime(x['date'],'%Y-%m-%d').date(), axis = 1)


# Calc totals for each fighter
which_fighter = ["f1","f2"]
stat_name_ending = ["kds","sigstrikes_l","sigstrikes_a","strikes_l","strikes_a","tds_l","tds_a","subs_a","passes",
                   "revs","ss_head_l","ss_head_a","ss_body_l","ss_body_a","ss_leg_l","ss_leg_a","ss_dist_l","ss_dist_a",
                   "ss_clinch_l","ss_clinch_a","ss_ground_l","ss_ground_a"]

for f in which_fighter:
    
    for stat in stat_name_ending:
        
        fs_df[f+'_t_' + stat] = np.sum(fs_df.loc[:,(fs_df.columns.str.startswith(f) & fs_df.columns.str.endswith(stat))], axis =1)
        
        fs_df[f+'_pm_' + stat] =  fs_df[f+'_t_' + stat] / fs_df['duration'] * 60


#### Subset the Data for Key Stats and Per Minute 

In [4]:
matchers = ['fighter','winner',"_pm_","date"]
ml_cols = [s for s in fs_df.columns if any(xs in s for xs in matchers)]
ml_df = fs_df[ml_cols]

#### Make each fight have two rows, so that the label is win or lose

In [5]:
# Part 1
part_1_df = ml_df.copy()

part1_key_cols = part_1_df.columns
part1_value_cols = []

for col in part1_key_cols:
    new_col = col.replace("f1","f")
    new_col = new_col.replace("f2","o")
    new_col = new_col.replace("fighter_1","a_fighter")
    new_col = new_col.replace("fighter_2","a_opponent")
    new_col = new_col.replace("winner","a_winner")
    part1_value_cols.append(new_col)
    
p1_col_dict = dict(zip(part1_key_cols, part1_value_cols))

part_1_df = part_1_df.rename(columns = p1_col_dict)

In [6]:
# Part 2
part_2_df = ml_df.copy()

part2_key_cols = part_2_df.columns
part2_value_cols = []

for col in part2_key_cols:
    new_col = col.replace("f1","o")
    new_col = new_col.replace("f2","f")
    new_col = new_col.replace("fighter_1","a_opponent")
    new_col = new_col.replace("fighter_2","a_fighter")
    new_col = new_col.replace("winner","a_winner")
    part2_value_cols.append(new_col)
    
p2_col_dict = dict(zip(part2_key_cols, part2_value_cols))

part_2_df = part_2_df.rename(columns = p2_col_dict)

In [7]:
two_pf = pd.concat([part_1_df, part_2_df], sort = True)
two_pf = two_pf.sort_values(by ='date', ascending= False)
two_pf['result'] = two_pf.apply(lambda x: int(x['a_fighter'] == x['a_winner']), axis =1)


#### Show head and describe the wrangled dataset

In [8]:
two_pf.head(3)

,a_fighter,a_opponent,a_winner,date,f_pm_kds,f_pm_passes,f_pm_revs,f_pm_sigstrikes_a,f_pm_sigstrikes_l,f_pm_ss_body_a,...,o_pm_ss_head_a,o_pm_ss_head_l,o_pm_ss_leg_a,o_pm_ss_leg_l,o_pm_strikes_a,o_pm_strikes_l,o_pm_subs_a,o_pm_tds_a,o_pm_tds_l,result
0,"Francis Ngannou ""The Predator""",Cain Velasquez,"Francis Ngannou ""The Predator""",2019-02-17,0.0,0.000000,0.0,46.153846,27.692308,2.307692,...,23.076923,13.846154,0.000000,0.000000,36.745562,12.248521,0.0,2.307692,0.0,1
1,"James Vick ""The Texecutioner""","Paul Felder ""The Irish Dragon""","James Vick ""The Texecutioner""",2019-02-17,0.0,0.000000,0.0,11.333333,5.533333,0.933333,...,5.000000,1.600000,1.666667,1.400000,25.328889,12.244444,0.0,0.000000,0.0,1
11,"Emily Whitmire ""Spitfire""","Alexandra Albu ""Stitch""","Alexandra Albu ""Stitch""",2019-02-17,0.0,1.967213,0.0,4.918033,2.950820,0.000000,...,8.852459,3.934426,0.983607,0.983607,41.150228,15.673206,0.0,0.000000,0.0,0


In [9]:
two_pf.describe()

,f_pm_kds,f_pm_passes,f_pm_revs,f_pm_sigstrikes_a,f_pm_sigstrikes_l,f_pm_ss_body_a,f_pm_ss_body_l,f_pm_ss_clinch_a,f_pm_ss_clinch_l,f_pm_ss_dist_a,...,o_pm_ss_head_a,o_pm_ss_head_l,o_pm_ss_leg_a,o_pm_ss_leg_l,o_pm_strikes_a,o_pm_strikes_l,o_pm_subs_a,o_pm_tds_a,o_pm_tds_l,result
count,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,...,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000,9866.000000
mean,0.128069,0.188637,0.020098,10.079072,4.672931,1.156535,0.806863,1.141474,0.779893,7.324719,...,8.062033,3.189895,0.861356,0.678066,37.308198,18.770931,0.111921,0.379623,0.162333,0.491790
std,0.681112,0.494391,0.180453,9.273370,5.734757,1.128777,0.879737,2.135073,1.491094,6.401251,...,6.997737,3.854575,0.991489,0.804398,46.824339,31.798594,0.427196,0.579660,0.374653,0.499958
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,4.705882,1.834179,0.400000,0.261059,0.133333,0.066667,2.733333,...,3.733333,1.200000,0.200000,0.133333,17.430650,7.481107,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,7.904192,3.333333,0.882353,0.600000,0.560748,0.343348,5.766711,...,6.333333,2.186867,0.600000,0.455812,27.600000,12.744637,0.000000,0.200000,0.000000,0.000000
75%,0.000000,0.200000,0.000000,12.631579,5.533333,1.590909,1.071429,1.333333,0.933333,10.133333,...,10.200000,3.733333,1.200000,0.963855,42.460000,20.077501,0.066667,0.533333,0.200000,1.000000
max,17.142857,13.333333,13.333333,137.142857,112.000000,21.000000,18.000000,51.176471,35.294118,75.000000,...,94.285714,56.250000,22.000000,16.000000,1016.326531,810.612245,13.333333,10.909091,10.909091,1.000000


# 2.2 Data Wrangling for fighter_char

#### Calculate mean height and reach to fill in missing data

In [10]:
# Height
height_df = fc_df[fc_df['height'] != "--"]
height_df = height_df.reset_index()

for index, row in height_df.iterrows():
    height_list = row['height'].split(" ")
    p1 = int(re.sub("[^0-9]", "", height_list[0]))*12
    p2 = int(re.sub("[^0-9]", "", height_list[1]))
    height_df.loc[index, 'height'] = p1 + p2
    
mean_height = np.mean(height_df['height'])

# Reach
reach_df = fc_df[fc_df['reach'] != "--"]
reach_df = reach_df.reset_index()

for index, row in reach_df.iterrows():
    reach_df.loc[index, 'reach'] = int(row['reach'].replace('"',""))
    
mean_reach = np.mean(reach_df['reach'])

In [11]:
fc_df = fc_df.drop(['Unnamed: 0'], axis = 1) # drops first column

fc_df = fc_df.fillna(value = "") # fills na

# Change values to numerics and add full_name column
for index, row in fc_df.iterrows(): 
    # add full_name
    if(row['nick_name'] == ""):
        fc_df.loc[index,'full_name'] = row['name']
        
    else:
        fc_df.loc[index,'full_name'] = row['name'] + ' "' + row['nick_name'] + '"' 
    
    # convert height to int(inches)
    if(row['height'] != "--"):
        height_list = row['height'].split(" ")
        fc_df.loc[index,'height'] = int(re.sub("[^0-9]", "", height_list[0]))*12 + int(re.sub("[^0-9]", "", height_list[1]))
    else:
        fc_df.loc[index,'height'] = mean_height
    
    # convert reach to date object
    if(row['reach'] != "--"):
        fc_df.loc[index,'reach'] = int(row['reach'].replace('"',""))
    else:
        fc_df.loc[index,'reach'] = mean_reach
        
    # convert dob to date object
    if(row['DOB'] != "--"):
        date_str = row['DOB'].replace(",","").replace(" ","-")
        fc_df.loc[index,'DOB'] = datetime.datetime.strptime(date_str, '%b-%d-%Y').date()
        
        

#### Show head and describe the wrangled dataset

In [12]:
fc_df.head(3)

,name,nick_name,fight_record,height,weight,reach,stance,DOB,full_name
0,Dongi Yang,The Ox,13-3-0,70,185 lbs.,71,Southpaw,1984-12-07,"Dongi Yang ""The Ox"""
1,Goldman Butler,,1-4-0,70.3936,--,71.8466,,--,Goldman Butler
2,Ruslan Magomedov,Leopard,14-1-0,75,242 lbs.,78,Orthodox,1986-11-26,"Ruslan Magomedov ""Leopard"""


In [13]:
fc_df.describe()

,name,nick_name,fight_record,height,weight,reach,stance,DOB,full_name
count,3214,3214,3214,3214.0,3214,3214.000000,3214,3214,3214
unique,3212,1393,1044,26.0,109,26.000000,6,2142,3214
top,Mike Davis,,0-1-0,72.0,170 lbs.,71.846561,Orthodox,--,"Andre Muniz ""Sergipano"""
freq,2,1492,75,370.0,506,1702.000000,1873,737,1


# 3. Create cumulative statistics data with specified look back window

#### Create functions to make the look back window dataframe

In [14]:
fighters_dict = generate_fighters_dict() # Generates a dictionary with each fighter having his own pd.DataFrame()

## Make cumulative data using 1,3,5,7,10 lookback window

In [15]:
exact_1_df = generate_cum_df(fighters_dict,1, True)
exact_4_df = generate_cum_df(fighters_dict,4, True)  
exact_7_df = generate_cum_df(fighters_dict,7, True)  
exact_10_df = generate_cum_df(fighters_dict,10, True)

cumu_1_df = generate_cum_df(fighters_dict,1, False)    
cumu_4_df = generate_cum_df(fighters_dict,4, False)      
cumu_7_df = generate_cum_df(fighters_dict,7, False)
cumu_10_df = generate_cum_df(fighters_dict,10, False)

In [16]:
cumu_dfs_dict = {"Exact Data: 1 Fight Lookback Window" : exact_1_df,
                 "Exact Data: 4 Fight Lookback Window" : exact_4_df,
                 "Exact Data: 7 Fight Lookback Window" : exact_7_df,
                 "Exact Data: 10 Fight Lookback Window" : exact_10_df,
                 "Cumulative Data: 1 Fight Lookback Window": cumu_1_df,
                 "Cumulative Data: 4 Fight Lookback Window": cumu_4_df,
                 "Cumulative Data: 7 Fight Lookback Window": cumu_7_df,
                 "Cumulative Data: 10 Fight Lookback Window": cumu_10_df} #  create dictionary of the dfs

# 4. Feature Engineering (Merging in height, reach, stance)

In [17]:
fe_cumu_dfs_dict = cumu_dfs_dict.copy() # create copy of aggregate data sets

accuracy_stats = ['sigstrikes', 'strikes', 'tds', 'ss_head', 'ss_body', 'ss_leg', 'ss_dist', 'ss_clinch', 'ss_ground']
for key in fe_cumu_dfs_dict:
    
    df = fe_cumu_dfs_dict[key].copy()
    
    
    for stat in accuracy_stats:
        
        fe_index = 0
        f_init = "fo"
        
        for letter in f_init:
            
            fe_index_r = fe_index%2
            # Merge in height, reach, and stance
            df['{0}_acu_{1}'.format(f_init[fe_index_r], stat)] =  df['{0}_pm_{1}_l'.format(f_init[fe_index_r], stat)] / df['{0}_pm_{1}_a'.format(f_init[fe_index_r], stat)]

            fe_index += 1
            
    df = df.fillna(0)
    fe_cumu_dfs_dict[key] = df
    

# 5. Calculate Differentials

# 6. Save data for Machine Learning 

In [18]:
data_dicts_list = [cumu_dfs_dict,fe_cumu_dfs_dict] # add dictionaries to list

#### Save the dictionaries

In [20]:
with open('../data/temp_data/data_dicts_v3.pickle', 'wb') as handle:
    pickle.dump(data_dicts_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
fe_cumu_lb1 = data_dicts_list[1]['Cumulative Data: 1 Fight Lookback Window']

In [32]:
fe_cumu_lb1.head(3)

,a_fighter,a_opponent,a_winner,date,f_age,f_height,f_pm_kds,f_pm_passes,f_pm_revs,f_pm_sigstrikes_a,...,f_acu_ss_body,o_acu_ss_body,f_acu_ss_leg,o_acu_ss_leg,f_acu_ss_dist,o_acu_ss_dist,f_acu_ss_clinch,o_acu_ss_clinch,f_acu_ss_ground,o_acu_ss_ground
0,"Francis Ngannou ""The Predator""",Cain Velasquez,"Francis Ngannou ""The Predator""",2019-02-17,32.0,76.0,0.300066,0.000000,0.000000,10.473689,...,0.684405,0.744228,0.554595,0.873359,0.351589,0.404840,0.416385,0.750656,0.441310,0.744366
0,"James Vick ""The Texecutioner""","Paul Felder ""The Irish Dragon""","James Vick ""The Texecutioner""",2019-02-17,31.0,75.0,0.095899,0.035425,0.024242,13.726264,...,0.629991,0.675825,0.626267,0.775490,0.331481,0.426990,0.712775,0.625749,0.735255,0.447881
0,"Emily Whitmire ""Spitfire""","Alexandra Albu ""Stitch""","Alexandra Albu ""Stitch""",2019-02-17,27.0,65.0,0.000000,0.033333,0.227273,9.509091,...,0.590909,0.867580,0.928571,0.975768,0.401807,0.471556,0.250000,0.883051,0.666667,0.650000


In [33]:
fe_cumu_lb1.columns

Index(['a_fighter', 'a_opponent', 'a_winner', 'date', 'f_age', 'f_height',
       'f_pm_kds', 'f_pm_passes', 'f_pm_revs', 'f_pm_sigstrikes_a',
       'f_pm_sigstrikes_l', 'f_pm_ss_body_a', 'f_pm_ss_body_l',
       'f_pm_ss_clinch_a', 'f_pm_ss_clinch_l', 'f_pm_ss_dist_a',
       'f_pm_ss_dist_l', 'f_pm_ss_ground_a', 'f_pm_ss_ground_l',
       'f_pm_ss_head_a', 'f_pm_ss_head_l', 'f_pm_ss_leg_a', 'f_pm_ss_leg_l',
       'f_pm_strikes_a', 'f_pm_strikes_l', 'f_pm_subs_a', 'f_pm_tds_a',
       'f_pm_tds_l', 'f_reach', 'f_stance', 'o_age', 'o_height', 'o_pm_kds',
       'o_pm_passes', 'o_pm_revs', 'o_pm_sigstrikes_a', 'o_pm_sigstrikes_l',
       'o_pm_ss_body_a', 'o_pm_ss_body_l', 'o_pm_ss_clinch_a',
       'o_pm_ss_clinch_l', 'o_pm_ss_dist_a', 'o_pm_ss_dist_l',
       'o_pm_ss_ground_a', 'o_pm_ss_ground_l', 'o_pm_ss_head_a',
       'o_pm_ss_head_l', 'o_pm_ss_leg_a', 'o_pm_ss_leg_l', 'o_pm_strikes_a',
       'o_pm_strikes_l', 'o_pm_subs_a', 'o_pm_tds_a', 'o_pm_tds_l', 'o_reach',
       'o_sta

# 7. Feature Selection? Use Johnson for reference